<a href="https://colab.research.google.com/github/JVCasas/ImersaoAlura2024Desafio/blob/main/DesafioAlura_ProjetoOrganizador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Organizador de pastas do drive**

_O objetivo do projeto é criar um programa que com o auxilio da Inteligência Artificial do Google, Gemini AI, faça a organização dos arquivos no Google Drive em pastas por tópicos fornecidos pelo usuário em um prompt._

**Aviso:** o programa irá fazer modificações dentro do Google Drive, por se tratar de um programa feito para estudos, **NÃO O UTILIZE EM UM GOOGLE DRIVE COM ARQUIVOS E CONTEÚDOS SENSÍVEIS, NÃO ME RESPONSABILIZO POR EVENTUAIS PERDAS, DANOS E/OU EXPOSIÇÕES.**

## 1- Instalando e Importando Bibliotecas Necessárias


In [1]:
pip install -u google-generativeai #Instalação do Google API


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [2]:
import google.generativeai as genai
from google.colab import drive, userdata
import os
import numpy
import pandas

## Setando a Chave API

In [3]:
chaveGoogleAPI = userdata.get('secret')
genai.configure(api_key = chaveGoogleAPI)

## Entrando no Google Drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


## Listando os arquivos na pasta a ser organizada

In [40]:
pasta = '/content/drive/MyDrive/OrganizeAqui/'
local = '/content/drive/MyDrive/OrganizeAqui'

lista = []

for n in os.listdir(local):
      destino = pasta + n
      arquive = open(destino, 'r')
      content = arquive.read()
      objeto = {"Titulo":n, "Conteudo":content}
      lista.append(objeto)

## Criando o Data Frame com a Lista

In [6]:
dfArquivos = pandas.DataFrame(lista)
dfArquivos

,Titulo,Conteudo
0,zxt444TBx3497ABac.txt,"Cactaceae é uma família botânica de arbustos, ..."
1,rtx49pzqu224.txt,"Cactaceae é uma família botânica de arbustos, ..."
2,Novo Documento de Texto.txt,"A dioneia, também conhecida como apanha-moscas..."
3,ttx4paiufaljgneaior.txt,"As samambaias, ou fetos, são vegetais vascular..."
4,afoajfojf.txt,O Dodge Viper é um super esportivo que começou...
5,akgoakrgvmavmk.txt,A Ferrari F40 é um automóvel superesportivo co...
6,Armadilha.txt,A planta baixa dos carros Dodge Viper e Ferrar...


## **Criando pastas**



Nesta etapa criamos um input que ira receber nomes de pastas e temas destas pastas que serão postas num data frame e serão gerados embeddings dos temas.

In [41]:
listaPasta = []
check = True
while check:
        nomePasta = input('Digite o nome da pasta:')
        nomeTema = input('Digite o tema da pasta:')
        directory = {"Nome":nomePasta, "Tema":nomeTema}
        listaPasta.append(directory)
        os.mkdir(pasta + directory['Nome'])
        while True:
                a = input('Deseja adicionar uma nova pasta? [S/N]')
                if a not in 'SsNn':
                        print('Digite apenas "S" ou "N".')
                else:
                        if a in 'Nn':
                               check = False
                               break
                        else:
                                break

dfPastas = pandas.DataFrame(listaPasta)
dfPastas

Digite o nome da pasta:Carros
Digite o tema da pasta:Carros e suas engenharias
Deseja adicionar uma nova pasta? [S/N]s
Digite o nome da pasta:Plantas
Digite o tema da pasta:Plantas e vegetais
Deseja adicionar uma nova pasta? [S/N]n


,Nome,Tema
0,Carros,Carros e suas engenharias
1,Plantas,Plantas e vegetais


In [42]:
os.listdir()

['.config', 'drive', 'sample_data']

In [37]:
os.rmdir('CarrosCarros')

In [43]:
print(pasta)

/content/drive/MyDrive/OrganizeAqui/


## Integrando o Gemini AI para criação de Embeddings

### Gerando lista dos modelos acessívei para uso

In [24]:
for m in genai.list_models():
      if 'embedContent' in m.supported_generation_methods:
            print(m.name)

models/embedding-001
models/text-embedding-004


### Criando Embeddings e Retornando no Data Frame

In [44]:
modelo = 'models/embedding-001'

def criaEmbed(conteudo):
      return genai.embed_content(model = modelo, content = conteudo, task_type="RETRIEVAL_DOCUMENT")["embedding"]

def criaEmbedArquivos(conteudo):
      return genai.embed_content(model = modelo, content = conteudo, task_type="RETRIEVAL_QUERY")["embedding"]

dfArquivos["Embed"] = dfArquivos.apply(lambda coluna: criaEmbedArquivos(coluna["Conteudo"]), axis = 1)
dfPastas["Embed"] = dfPastas.apply(lambda coluna: criaEmbed(coluna["Tema"]), axis = 1)

In [45]:
dfArquivos

,Titulo,Conteudo,Embed
0,zxt444TBx3497ABac.txt,"Cactaceae é uma família botânica de arbustos, ...","[0.028230222, -0.00091631024, -0.06080193, 0.0..."
1,rtx49pzqu224.txt,"Cactaceae é uma família botânica de arbustos, ...","[0.028230222, -0.00091631024, -0.06080193, 0.0..."
2,Novo Documento de Texto.txt,"A dioneia, também conhecida como apanha-moscas...","[0.056948237, -0.05044992, 0.0015834471, 0.019..."
3,ttx4paiufaljgneaior.txt,"As samambaias, ou fetos, são vegetais vascular...","[0.040045764, -0.045055494, -0.036180798, 0.02..."
4,afoajfojf.txt,O Dodge Viper é um super esportivo que começou...,"[-0.044938255, -0.050740793, -0.021967722, 0.0..."
5,akgoakrgvmavmk.txt,A Ferrari F40 é um automóvel superesportivo co...,"[0.030469526, -0.03418814, -0.0112952, 0.00277..."
6,Armadilha.txt,A planta baixa dos carros Dodge Viper e Ferrar...,"[-0.008144178, -0.034073118, -0.030584931, -0...."


In [46]:
dfPastas

,Nome,Tema,Embed
0,Carros,Carros e suas engenharias,"[0.0181842, -0.0023732102, -0.060256075, 0.007..."
1,Plantas,Plantas e vegetais,"[0.03279264, -0.009128104, -0.0549767, -0.0021..."


### Criando função para buscador de temas

In [47]:
import shutil

In [50]:
#Nova função de busca
for n in dfArquivos.iloc():
      produtoEscalar = numpy.dot(numpy.stack(dfPastas["Embed"]), n['Embed'])
      indice = numpy.argmax(produtoEscalar)
      arquivo = n['Titulo']
      nPastas = dfPastas.iloc[indice]["Nome"]
      shutil.move(pasta + n['Titulo'], pasta + nPastas)
